In [ ]:
!python -V

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, roc_auc_score, log_loss
from sklearn.model_selection import train_test_split

In [ ]:
data_both_cohorts = pd.read_csv('data/Debernardi et al 2020 data.csv')
data = data_both_cohorts[data_both_cohorts['patient_cohort'] == 'Cohort1'] # Keep Cohort2 for later

In [ ]:
data['diagnosis'] = data['diagnosis'].apply(lambda d: d-1)
dummies = pd.get_dummies(data['sex'], drop_first=True)
data = pd.concat([data.drop('sex', axis=1), dummies], axis=1)

In [ ]:
categorical = ['M']
numerical = ['age','creatinine', 'LYVE1', 'REG1B', 'TFF1']
target = ['diagnosis']

In [ ]:
X = data[categorical + numerical].values
y = np.squeeze(data[target].values) # Squeeze cause of dimension errors in XGBoost

In [ ]:
# Let the CV algorithm perform the training - validation split implicitly
#X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

# Perform the train - test - validation split here already
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.8)
X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=0.5)

In [ ]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

In [71]:
def objective(params):
    booster = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=1000,
        evals=[(dval, 'validation')],
        early_stopping_rounds=50
    )
    y_pred = booster.predict(dval)
    print(f'y_val = {y_val}')

    print(f'y_pred = {y_pred}')

    #return {'loss': log_loss(y_val, y_pred), 'status': STATUS_OK}
    return {'loss': -roc_auc_score(y_val, y_pred, multi_class='ovr'), 'status': STATUS_OK}

In [72]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 15, 1)),
    'subsample': hp.uniform('subsample', .5, 1.0),
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 7),
    'reg_alpha': hp.loguniform('reg_alpha', -10, 10),
    'reg_lambda': hp.loguniform('reg_lambda', -10, 10),
    'gamma': hp.loguniform('gamma', -10, 10),
    'objective': 'multi:softprob',
    #'eval_metric': 'mlogloss',
    'eval_metric': 'auc',
    'num_class': 3
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-auc:0.50000                            
[1]	validation-auc:0.50000                            
[2]	validation-auc:0.50000                            
[3]	validation-auc:0.50000                            
[4]	validation-auc:0.50000                            
[5]	validation-auc:0.50000                            
[6]	validation-auc:0.50000                            
[7]	validation-auc:0.50000                            
[8]	validation-auc:0.50000                            
[9]	validation-auc:0.50000                            
[10]	validation-auc:0.50000                           
[11]	validation-auc:0.50000                           
[12]	validation-auc:0.50000                           
[13]	validation-auc:0.50000                           
[14]	validation-auc:0.50000                           
[15]	validation-auc:0.50000                           
[16]	validation-auc:0.50000                           
[17]	validation-auc:0.50000                           
[18]	valid

In [78]:
best_result['max_depth'] = int(best_result['max_depth'])

In [80]:
best_model = xgb.train(
    params=best_result,
    dtrain=dtrain,
    num_boost_round=1000)

In [88]:
y_pred = best_model.predict(xgb.DMatrix(X_test, label=y_test))
predictions = [round(value) for value in y_pred]

In [96]:
from sklearn.metrics import f1_score
test_score = f1_score(y_test, predictions, average='weighted')
conf_matrix = confusion_matrix(y_test, predictions)

In [95]:
test_score

0.5530204000792236